# x. Model Inference

>Model yang sudah dilatih akan dicoba pada data yang bukan termasuk ke dalam train-set, val-set, ataupun test-set. Data ini harus dalam format yang asli, bukan data yang sudah di-scaled.

Pada bagian ini akan dilakukan proses model inference dengan menggunakan data `inference set` dan melakukan preprocessing yang sama seperti yang dilakukan sebelumnya serta melakukan prediksi terhadap data `inference-set`.

## Import Libraries

In [ ]:
## REQUIRED LIBRARIES

# For load model
from tensorflow.keras.models import load_model

# For data wrangling
import numpy as np  # linear algebra
import pandas as pd  # data processing

# for model inference
from tensorflow.keras.preprocessing.image import ImageDataGenerator

## Load model

In [ ]:
# load best model

best_model = load_model('model.h5')
best_model

## Load data

In [ ]:
# create a folder in kaggle root
!mkdir -p ~/.kaggle
# copy paste kaggle.json to the kaggle folder
!cp kaggle.json ~/.kaggle

# create kaggle folder in same location (not in root)
!mkdir ./kaggle
# move the kaggle.json file to the created kaggle folder
!mv ./kaggle.json ./kaggle

In [ ]:
# change access permissions in a directory, only the owner of the file has read and write access to the file
!chmod 600 ~/.kaggle/kaggle.json

# check the list of data in the kaggle folder
!ls ~/.kaggle 

kaggle.json


In [ ]:
# download dataset
!kaggle datasets download -d rayhanzamzamy/non-and-biodegradable-waste-dataset

 99% 1.85G/1.86G [00:20<00:00, 128MB/s]
100% 1.86G/1.86G [00:20<00:00, 97.2MB/s]


In [ ]:
# unzip the downloaded folder from Kaggle
!unzip -q /content/non-and-biodegradable-waste-dataset.zip -d images

In [ ]:
# load data
df_inf = pd.read_csv('/content/data inference.csv')
df_inf

,images,label
0,/content/images/TRAIN.1/N/TRAIN.1_NBIODEG_HFL_...,N
1,/content/images/TRAIN.1/N/TRAIN.1_NBIODEG_CWR_...,N
2,/content/images/TRAIN.1/N/TRAIN.1_NBIODEG_CWR_...,N
3,/content/images/TRAIN.1/N/TRAIN.1_NBIODEG_VFL_...,N
4,/content/images/TRAIN.1/B/TRAIN.1_BIODEG_ORI_1...,B
5,/content/images/TRAIN.1/N/TRAIN.1_NBIODEG_ORI_...,N
6,/content/images/TRAIN.1/N/TRAIN.1_NBIODEG_CCW_...,N
7,/content/images/TRAIN.1/B/TRAIN.1_BIODEG_ORI_2...,B
8,/content/images/TRAIN.1/B/TRAIN.1_BIODEG_ORI_2...,B
9,/content/images/TRAIN.1/B/TRAIN.1_BIODEG_ORI_1...,B


## Feature Engineering

In [ ]:
# fitur
X = 'images'
# target
y = 'label'

In [ ]:
# define height and width of image
height_img = 180
width_img = 180

# define random number
SEED = 23

# define batch
BATCH = 512

In [ ]:
# rescaling
inf_datagen= ImageDataGenerator(rescale= 1./255)

# include it in the inference dataframe
ds_inf= inf_datagen.flow_from_dataframe(
    dataframe= df_inf,
    x_col= X,
    y_col= y,
    target_size= (height_img,width_img),
    class_mode= 'binary',
    batch_size= BATCH,
    shuffle= False,
    seed= SEED)

Found 10 validated image filenames belonging to 2 classes.


In [ ]:
# Check the label and class of the label

inf_class = ds_inf.class_indices
print('Inference Set Class Label : ', inf_class)

Inference Set Class Label :  {'B': 0, 'N': 1}


## Model Prediction

In [ ]:
# make predictions
y_pred_inf = best_model.predict(ds_inf)
# set the threshold for classification
y_pred_inf = np.where(y_pred_inf >= 0.5, 1, 0)

# make the prediction results into a dataframe
y_pred_inf_df = pd.DataFrame(y_pred_inf, columns=['label Prediction'])
# value conversion
y_pred_inf_df['label Prediction'].replace(to_replace=0, value='B', inplace=True)
y_pred_inf_df['label Prediction'].replace(to_replace=1,  value='N', inplace=True)
     

1/1 [==============================] - 3s 3s/step


In [ ]:
# concate inference data with prediction result
data_inf_predict = pd.concat([df_inf, y_pred_inf_df], axis=1)
data_inf_predict

,images,label,label Prediction
0,/content/images/TRAIN.1/N/TRAIN.1_NBIODEG_HFL_...,N,N
1,/content/images/TRAIN.1/N/TRAIN.1_NBIODEG_CWR_...,N,N
2,/content/images/TRAIN.1/N/TRAIN.1_NBIODEG_CWR_...,N,N
3,/content/images/TRAIN.1/N/TRAIN.1_NBIODEG_VFL_...,N,N
4,/content/images/TRAIN.1/B/TRAIN.1_BIODEG_ORI_1...,B,B
5,/content/images/TRAIN.1/N/TRAIN.1_NBIODEG_ORI_...,N,N
6,/content/images/TRAIN.1/N/TRAIN.1_NBIODEG_CCW_...,N,N
7,/content/images/TRAIN.1/B/TRAIN.1_BIODEG_ORI_2...,B,B
8,/content/images/TRAIN.1/B/TRAIN.1_BIODEG_ORI_2...,B,B
9,/content/images/TRAIN.1/B/TRAIN.1_BIODEG_ORI_1...,B,B


Berdasarkan dataFrame di atas, kita dapat mengetahui bagaimana keakuratan model mengklasifikasikan gambar apakah limbah termasuk material Biodegradable atau Non-biodegradable. Kita dapat melihat bahwa tidak ada perbedaan antara hasil prediksi dengan data aktual. Artinya, model ini sudah bagus untuk mengklasifikasikan limbah termasuk material Biodegradable atau Non-biodegradable dan bisa digunakan oleh beberapa negara untuk memaksimalkan pengelolaan limbah.<br>
Hal ini dapat dibuktikan juga pada proses evaluasi model sebelumnya, kita mendapatkan bahwa model ini memiliki performansi terbaik mencapai `akurasi 95-98%` setelah dilakukan improvement.